In [1]:
import importlib
import argparse
import os, sys
import argparse
import pandas as pd
import numpy as np
import pickle
from math import sqrt

In [2]:
from sklearn.preprocessing import Imputer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import accuracy_score, log_loss

In [3]:
sys.path.insert(0, './../utils/')
from utils import *

In [4]:
#Define parser
#name = 'bpi_2012'
#name = 'bpi_2013'
name = 'small_log'
#name = 'large_log' 

parser = {
    'data_file': name + '.csv',
    'nan_pct': 0.3,
    'input_dir': '../input/{}/'.format(name), 
    'output_dir': './output/{}/'.format(name),
}

args = argparse.Namespace(**parser)

In [5]:
file_name = os.path.join(args.input_dir, 'parameters_{}.pkl'.format(args.nan_pct))
with open(file_name, 'rb') as f:
    most_frequent_activity = pickle.load(f)
    first_timestamp = pickle.load(f)
    avai_instance = pickle.load(f)
    nan_instance = pickle.load(f)
    train_size = pickle.load(f)
    val_size = pickle.load(f)
    test_size = pickle.load(f)
    train_row_num = pickle.load(f)
    val_row_num = pickle.load(f)
    test_row_num = pickle.load(f)

# Load data

In [6]:
normalized_complete_df_name = os.path.join(args.input_dir, 'normalized_complete_df_{}.csv'.format(args.nan_pct))
normalized_complete_df = pd.read_csv(normalized_complete_df_name)

normalized_missing_df_name = os.path.join(args.input_dir, 'normalized_missing_df_{}.csv'.format(args.nan_pct))
normalized_missing_df = pd.read_csv(normalized_missing_df_name)

In [7]:
missing_true_test = normalized_missing_df[-test_row_num:].reset_index(drop=True)
complete_true_test = normalized_complete_df[-test_row_num:].reset_index(drop=True)

In [8]:
complete_true_test['CompleteTimestamp'] = pd.to_datetime(complete_true_test['CompleteTimestamp'], errors='coerce')

In [9]:
complete_true_test.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,1601,Activity A,1970-01-01 09:00:00,0.0,0.000000
1,1601,Activity B,1970-01-01 10:00:00,3600.0,0.125000
2,1601,Activity C,1970-01-01 11:00:00,7200.0,0.249999
3,1601,Activity D,1970-01-01 12:00:00,10800.0,0.374999
4,1601,Activity E,1970-01-01 13:00:00,14400.0,0.499998


In [10]:
groupByCase = complete_true_test.groupby(['CaseID'])

In [11]:
duration_list = []
for caseid, data_case in groupByCase:
    for row in range(data_case.shape[0]):
        if row == 0:
            duration_case = 0
        else:
            current_time = data_case.iloc[row, data_case.columns.get_loc('CompleteTimestamp')]
            previous_time = data_case.iloc[row - 1, data_case.columns.get_loc('CompleteTimestamp')]
            duration_case = (current_time - previous_time).total_seconds()
        duration_list.append(duration_case)

In [12]:
len(duration_list) == complete_true_test.shape[0]

True

In [13]:
complete_true_test['Duration'] = duration_list

In [14]:
total_duration_list = []
for caseid, data_case in groupByCase:
    last_time = data_case.iloc[-1, data_case.columns.get_loc('CompleteTimestamp')]
    first_time = data_case.iloc[0, data_case.columns.get_loc('CompleteTimestamp')]
    total_duration_case = (last_time - first_time).total_seconds()
    total_duration_list.append(total_duration_case)

In [15]:
len(total_duration_list) == len(groupByCase)

True

In [16]:
complete_true_test.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime,Duration
0,1601,Activity A,1970-01-01 09:00:00,0.0,0.000000,0.0
1,1601,Activity B,1970-01-01 10:00:00,3600.0,0.125000,3600.0
2,1601,Activity C,1970-01-01 11:00:00,7200.0,0.249999,3600.0
3,1601,Activity D,1970-01-01 12:00:00,10800.0,0.374999,3600.0
4,1601,Activity E,1970-01-01 13:00:00,14400.0,0.499998,3600.0


In [17]:
complete_true_test['Duration'].describe()

count    5600.000000
mean     2057.142857
std      1781.697061
min         0.000000
25%         0.000000
50%      3600.000000
75%      3600.000000
max      3600.000000
Name: Duration, dtype: float64

In [18]:
normalized_complete_df['Activity'].value_counts()

Activity M    2000
Activity C    2000
Activity G    2000
Activity H    2000
Activity B    2000
Activity L    2000
Activity K    2000
Activity J    2000
Activity I    2000
Activity E    2000
Activity D    2000
Activity A    2000
Activity F    2000
Activity N    2000
Name: Activity, dtype: int64